!pip install azure-storage-file-datalake

Install pyarrowfs_adlgen2. It is isn't available in internal package manager so can't add it to requirements.txt

In [1]:
%pip install pyarrowfs_adlgen2

In [2]:
from azure.core.credentials import AccessToken
from notebookutils import mssparkutils
import pyarrow.fs
import pyarrowfs_adlgen2
from azure.identity import ManagedIdentityCredential
from datetime import datetime

class SynapseAuthToken:
    def get_token(*args, **kwargs):
        # credential = ManagedIdentityCredential()
        # credential._credential = SynapseAuthToken()
        dt = datetime.today()
        seconds = dt.timestamp()

        return AccessToken(
            token=mssparkutils.credentials.getToken(audience="storage"),
            # some random time in the future... synapse doesn't provide way to get an expires_on value for a token
            expires_on=seconds + (60 * 100)
        )


def __get_azure_filesystem(storage_acc_name, container_name):
    credential = ManagedIdentityCredential()
    credential._credential = SynapseAuthToken()

    # Use the credential from spoof_token class
    handler = FilesystemHandler.from_account_name(
        storage_acc_name, container_name, credential)

    fs: FilesystemHandler = pq.fs.PyFileSystem(handler)

def get_parquet_metadata(file_path, storage=Storage.Local, storage_acc_name=None, container_name=None):
    if storage == Storage.Local:
        pq_file: ParquetFile = pq.ParquetFile(file_path)
    else:
        file_sys_handler = __get_azure_filesystem(storage_acc_name, container_name)
        pq_file: ParquetFile = pq.ParquetFile(file_path, filesystem=file_sys_handler)

    # Return the metadata components as a tuple
    return (
        get_file_meta(pq_file),
        get_schema(pq_file),
        get_row_groups_meta(pq_file),
        get_columns_meta(pq_file),
        get_columns_stats(pq_file)
    )


import pkg_resources

installed_packages = pkg_resources.working_set
for package in sorted([f"{i.key}=={i.version}" for i in installed_packages]):
    print(package)

In [3]:
from metadata_utils import get_parquet_metadata, get_parquet_metadata_json, get_parquet_metadata_df

def print_parquet_metadata(file_path, print_json=False, print_dataframe=True):
    file_summary, schema, row_groups, columns, column_stats = get_parquet_metadata(file_path)
    if print_json:
        file_summary_json, schema_json, row_groups_json, columns_json, column_stats_json = get_parquet_metadata_json(
            file_path)
    if print_dataframe:
        file_summary_df, schema_df, row_groups_df, columns_df, column_stats_df = get_parquet_metadata_df(file_path)

    # 1. File summary
    print("\nFile Summary:")
    print("\t", file_summary_json) if print_json else None
    print(file_summary_df) if print_dataframe else None

    #  2. Schema
    print("\nSchema:")
    print("\t", schema_json) if print_json else None
    print(schema_df) if print_dataframe else None

    # 3. Row group summary
    print("\nRow Groups Summary: ", len(row_groups))
    print("\t", row_groups_json) if print_json else None
    print(row_groups_df) if print_dataframe else None

    # 4. Column metadata
    print("\nColumn Summary: ")
    print("\t", columns_json) if print_json else None
    print(columns_df) if print_dataframe else None

    # 5.  Column stats
    print("\nColumn Statistics: ")
    print("\t", column_stats_json) if print_json else None
    print(column_stats_df) if print_dataframe else None

In [1]:
#pq_path = '/Users/o60774/Documents/WS/datasets/new_york_taxi/parquet/'
pq_path='abfss://tlfs@xxx.dfs.core.windows.net/poc/parquet/nyc_yellow_taxi_trips/puYear=2001/puMonth=1/part-00493-127a558e-137b-4f6b-8c6e-904dc74a264e.c000.snappy.parquet'

import io
from azure.storage.blob import BlockBlobService
import pyarrow.parquet as pq

# Replace these variables with your actual account_name and account_key
ACCOUNT_NAME = "xxx"
ACCOUNT_KEY = ""

# Replace this with the container name and Parquet file path in ADLS Gen2
CONTAINER_NAME = "tlfs"
PARQUET_FILE_PATH = 'poc/parquet/nyc_yellow_taxi_trips/puYear=2001/puMonth=1/part-00493-127a558e-137b-4f6b-8c6e-904dc74a264e.c000.snappy.parquet'

# Create a BlockBlobService instance to interact with blob storage
block_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# Read the Parquet file as a stream
with io.BytesIO() as parquet_stream:
    block_blob_service.get_blob_to_stream(CONTAINER_NAME, PARQUET_FILE_PATH, parquet_stream)
    parquet_stream.seek(0)

    # Open the Parquet file using ParquetFile class to get metadata
    parquet_file = pq.ParquetFile(parquet_stream)
    
    # Retrieve and print metadata
    metadata = parquet_file.metadata
    print("Metadata:\n", metadata)



#print_parquet_metadata(pq_path + 'part-00113.snappy.parquet', print_dataframe=True)

In [2]:
# Replace these variables with your actual account_name and account_key
ACCOUNT_NAME = "xxx"
ACCOUNT_KEY = ""

# Replace this with the container name and Parquet file path in ADLS Gen2
CONTAINER_NAME = "tlfs"
PARQUET_FILE_PATH = 'poc/parquet/nyc_yellow_taxi_trips/puYear=2001/puMonth=1/part-00493-127a558e-137b-4f6b-8c6e-904dc74a264e.c000.snappy.parquet'


In [3]:
from azure.identity import DefaultAzureCredential
import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [4]:
from azure.core.credentials import AzureNamedKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.storage.filedatalake import (
    AnalyticsLogging,
    CorsRule,
    DataLakeDirectoryClient,
    DataLakeFileClient,
    DataLakeServiceClient,
    FileSystemClient,
    Metrics,
    RetentionPolicy,
    StaticWebsite)

In [5]:
account_url = f"https://{ACCOUNT_NAME}.blob.core.windows.net"

try:
    print("Azure Blob Storage Python quickstart sample")

    default_credential = DefaultAzureCredential()
    # Create the BlobServiceClient object
    blob_service_client = BlobServiceClient(account_url, credential=default_credential)
    #stream = blob_service_client.download_blob()

except ResourceNotFoundError:
    print("No blob found.")
except Exception as ex:
    print('Exception:')
    print(ex)

In [11]:
service_client = DataLakeServiceClient.from_connection_string(
        f"DefaultEndpointsProtocol=https;AccountName={ACCOUNT_NAME};AccountKey={ACCOUNT_KEY};EndpointSuffix=core.windows.net")

file_system_client = service_client.get_file_system_client(file_system=CONTAINER_NAME)

directory_client = file_system_client.get_directory_client("poc/parquet/nyc_yellow_taxi_trips/puYear=2001/puMonth=1")

file_client = directory_client.get_file_client("part-00493-127a558e-137b-4f6b-8c6e-904dc74a264e.c000.snappy.parquet")

download = file_client.download_file()

downloaded_bytes = download.readall()

#with open("./sample.txt", "wb") as my_file:
#    my_file.write(downloaded_bytes)
#    my_file.close()

In [12]:
from azure.storage.filedatalake import DataLakeFileClient

file = DataLakeFileClient.from_connection_string(f"DefaultEndpointsProtocol=https;AccountName={ACCOUNT_NAME};AccountKey={ACCOUNT_KEY};EndpointSuffix=core.windows.net",
                                                 file_system_name=CONTAINER_NAME, file_path="poc/parquet/nyc_yellow_taxi_trips/puYear=2001/puMonth=1")

with open("./part-00493-127a558e-137b-4f6b-8c6e-904dc74a264e.c000.snappy.parquet", "wb") as my_file:
    download = file.download_file()
    download.readinto(my_file)